In [1]:
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [2]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard


2.15.0


In [3]:
X = X/255.0 # normalized data

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1)

In [5]:
print(y_val)

[1 3 0 0 0 0 2 4 1 2 2 1 3 4 0 2 1 2 3 1 4 1 4 0 4 3 4 2]


In [6]:
activation = 'relu'
feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = X.shape[1:]))

feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Flatten())

In [7]:
x = feature_extractor.output  
x = Dense(128, activation = activation, kernel_initializer = 'he_uniform')(x)
prediction_layer = Dense(5, activation = 'softmax')(x)

In [8]:
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary()) 


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 64)        36928     
                                                            

In [9]:
cnn_model.fit(X, y, epochs=30)

Epoch 1/30


9/9 [==============================] - 6s 495ms/step - loss: 5.2306 - accuracy: 0.2238
Epoch 2/30
9/9 [==============================] - 5s 524ms/step - loss: 1.4278 - accuracy: 0.3285
Epoch 3/30
9/9 [==============================] - 5s 548ms/step - loss: 1.0265 - accuracy: 0.6462
Epoch 4/30
9/9 [==============================] - 5s 531ms/step - loss: 0.7046 - accuracy: 0.7220
Epoch 5/30
9/9 [==============================] - 5s 502ms/step - loss: 0.8420 - accuracy: 0.6498
Epoch 6/30
9/9 [==============================] - 5s 507ms/step - loss: 0.5313 - accuracy: 0.7653
Epoch 7/30
9/9 [==============================] - 4s 493ms/step - loss: 0.3296 - accuracy: 0.8484
Epoch 8/30
9/9 [==============================] - 4s 490ms/step - loss: 0.2892 - accuracy: 0.8881
Epoch 9/30
9/9 [==============================] - 5s 506ms/step - loss: 0.3658 - accuracy: 0.8484
Epoch 10/30
9/9 [==============================] - 5s 557ms/step - loss: 0.1902 - accuracy: 0.9458
Epoch 11/30
9/9 [

In [10]:
x_train_RF = feature_extractor.predict(X)

9/9 [==============================] - 1s 75ms/step


In [11]:
print(x_train_RF.shape)

(277, 65536)


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [13]:
parameters = {'n_estimators': (10,20,30,40,50,70), 'criterion': ('gini', 'entropy'), 'max_depth': (3,5,7),
             'max_features' : (2,4), 'min_samples_split' : (2,4)}

In [14]:
RF_grid = GridSearchCV(RandomForestClassifier(n_jobs=-1, oob_score=False), param_grid=parameters, cv=3, verbose=True)

In [15]:
RF_grid_model = RF_grid.fit(x_train_RF, y)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


In [16]:
RF_grid_model.best_estimator_

RandomForestClassifier(max_depth=7, max_features=2, n_estimators=70, n_jobs=-1)

In [17]:
RF_grid_model.best_score_

0.8556178899797414

In [20]:
RF_model = RandomForestClassifier(max_depth=7, max_features=2, n_estimators=70, n_jobs=-1)

In [21]:
RF_model.fit(x_train_RF, y)

RandomForestClassifier(max_depth=7, max_features=2, n_estimators=70, n_jobs=-1)

In [ ]:
#x_test_RF = feature_extractor.predict(X_val)

In [ ]:
#predictions_RF = RF_model.predict(x_test_RF)
#predictions = cnn_model.predict(X_val)

In [ ]:
#import numpy as np
#pred_classes = []
#for pred in predictions:
    #pred_classes.append(np.argmax(pred))
#pred_classes = np.array(pred_classes)
#print("PREDICTIONS CNN_MODEL: ")
#print(pred_classes)
#print("TRUE VALUES: ")
#print(y_val)
#print("PREDICTIONS RANDOM_FOREST: ")
#print(predictions_RF)

In [ ]:
#wrong_predictions = 0
#for i in range(len(y_val)):
    #if y_val[i] != predictions_RF[i]:
        #print(f"TRUE: {y_val[i]} PRED: {predictions_RF[i]}")
        #wrong_predictions += 1

In [ ]:
#RF_accuracy = 1 - (wrong_predictions/len(y_val))
#print(f"Random Forest accuracy: {RF_accuracy}")

In [ ]:
#cnn_model.save('CNN_leafs.model')

In [26]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [27]:
feature_extractor.save('feature_extractor_model.h5')
from joblib import dump, load
dump(RF_model, 'RF_model.joblib')

C:\Users\adria\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['RF_model.joblib']